## Let's work with IoT devices data

In [0]:
iot_path = "/databricks-datasets/iot/iot_devices.json"
iot_df = spark.read.json(iot_path)

display(iot_df.limit(5))

from pyspark.sql.functions import col

# filter those temp > 0
hot_devices = iot_df.filter(col("temp") > 0) \
    .select("cn", "temp", "device_name")


# make sql know this
hot_devices.createOrReplaceTempView("iot_data")

summary_report = spark.sql("""
    SELECT 
        cn as Country,
        AVG(temp) as Avg_Temp,
        MAX(temp) as Max_Temp
    FROM iot_data
    GROUP BY 1
    ORDER BY Max_Temp DESC
""")

display(summary_report.limit(5))

In [0]:
# save the result to s3 bucket!

s3_path = "s3://wj-spark-bucket/iot_project/summary"
summary_report.write.format("parquet").mode("overwrite").save(s3_path)